# Get Datasets

In [1]:
import os

if not os.path.isdir('./data'):
    !mkdir ./data

if not os.path.isdir('./data/cora'):
    !mkdir ./data/cora
    !curl -o ./data/cora/cora.tar.gz "https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz"
    !gzip -d < ./data/cora/cora.tar.gz | tar xf - --directory ./data
    !rm ./data/cora/cora.tar.gz
    
if not os.path.isdir('./data/blogcatalog'):
    !mkdir ./data/blogcatalog
    !curl -o ./data/blogcatalog/b.zip "http://socialcomputing.asu.edu/uploads/1283153973/BlogCatalog-dataset.zip"
    !unzip -d ./data/blogcatalog ./data/blogcatalog/b.zip
    !mv ./data/blogcatalog/BlogCatalog-dataset/data/* ./data/blogcatalog/
    !mv ./data/blogcatalog/BlogCatalog-dataset/readme.txt ./data/blogcatalog/readme.txt
    # replace ',' by ' ' as delimiter
    !sed -i 's/,/ /g' ./data/blogcatalog/edges.csv
    !sed -i 's/,/ /g' ./data/blogcatalog/group-edges.csv
    !rm ./data/blogcatalog/b.zip
    !rm -r ./data/blogcatalog/BlogCatalog-dataset/    
    
if not os.path.isdir('./data/citeseer'):
    !mkdir ./data/citeseer
    !curl -o ./data/citeseer/c.tgz "https://linqs-data.soe.ucsc.edu/public/lbc/citeseer.tgz"
    !gzip -d < ./data/citeseer/c.tgz | tar xf - --directory ./data
    !rm ./data/citeseer/c.tgz
    
# https://snap.stanford.edu/data/egonets-Facebook.html
if not os.path.isdir('./data/facebook'):
    !mkdir ./data/facebook
    !curl -o ./data/facebook/facebook.gz "https://snap.stanford.edu/data/facebook_combined.txt.gz"
    !gzip -d ./data/facebook/facebook.gz
    !mv ./data/facebook/facebook ./data/facebook/facebook.edges
    
# https://snap.stanford.edu/data/com-Youtube.html
if not os.path.isdir('./data/youtube'):
    !mkdir ./data/youtube
    !curl -o ./data/youtube/youtube.gz "https://snap.stanford.edu/data/bigdata/communities/com-youtube.ungraph.txt.gz"
    !gzip -d ./data/youtube/youtube.gz
    !mv ./data/youtube/youtube ./data/youtube/youtube.edges
    # remove header (first 4 lines)
    !sed -i '1,4d' ./data/youtube/youtube.edges

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  163k  100  163k    0     0   136k      0  0:00:01  0:00:01 --:--:--  136k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  954k  100  954k    0     0   251k      0  0:00:03  0:00:03 --:--:--  251k
Archive:  ./data/blogcatalog/b.zip
   creating: ./data/blogcatalog/BlogCatalog-dataset/
   creating: ./data/blogcatalog/BlogCatalog-dataset/data/
  inflating: ./data/blogcatalog/BlogCatalog-dataset/data/edges.csv  
  inflating: ./data/blogcatalog/BlogCatalog-dataset/data/group-edges.csv  
  inflating: ./data/blogcatalog/BlogCatalog-dataset/data/groups.csv  
  inflating: ./data/blogcatalog/BlogCatalog-dataset/data/nodes.csv  
  inflating: ./data/blogcatalog/BlogCatalog-dataset/readme.txt  
  % Total    % Received % Xferd 

# Preprocess Datasets
Replace node-IDs with own ids, starting at 0. Remove self-loops and isolates (only present in citeseer). Only keep edgelists and label assignments (document content not needed).

In [2]:
import networkx as nx

if not os.path.isdir('./data/processed'):
    !mkdir ./data/processed
if not os.path.isdir('./data/emb'):
    !mkdir ./data/emb
    
datapath = './data/'
datasets = {
    'cora':{'edges':'cora.cites','labels':'cora.content'},
    'blogcatalog':{'edges':'edges.csv','labels':'group-edges.csv'},
    'citeseer':{'edges':'citeseer.cites','labels':'citeseer.content'}
}
for d in datasets:
    node2idx = dict()
    g = nx.read_edgelist(datapath + d + '/' + datasets[d]['edges'])
    g.remove_edges_from(g.selfloop_edges()) # remove self-loops
    g.remove_nodes_from(list(nx.isolates(g))) # remove isolates
    for n in g.nodes():
        node2idx[n] = str(len(node2idx))
    with open(datapath + 'processed/' + d + '.edges','w') as f:
        for e in g.edges():
            f.write(node2idx[e[0]] + '\t' + node2idx[e[1]] + '\n')
    with open(datapath + d + '/' + datasets[d]['labels']) as f_in:
        with open(datapath + 'processed/' + d + '.labels','w') as f_out:
            for line in f_in.readlines():
                line = line.split()
                if line[0] in node2idx:
                    f_out.write(node2idx[line[0]] + '\t' + line[-1] + '\n')

In [3]:
datasets = ['facebook','youtube']
for d in datasets:
    node2idx = dict()
    g = nx.read_edgelist(datapath + d + '/' + d + '.edges')
    g.remove_edges_from(g.selfloop_edges()) # remove self-loops
    g.remove_nodes_from(list(nx.isolates(g))) # remove isolates
    for n in g.nodes():
        node2idx[n] = str(len(node2idx))
    with open(datapath + 'processed/' + d + '.edges','w') as f:
        for e in g.edges():
            f.write(node2idx[e[0]] + '\t' + node2idx[e[1]] + '\n')

# Train Embeddings

In [2]:
%%time

from deepwalk import graph
import halk
from gensim.models import Word2Vec
import walklets
import node2vec
import networkx as nx
from HP import harp
import numpy as np

data_path = './data/processed/'
emb_path = './data/emb/'
# loop over datasets
datasets = ['cora','citeseer','blogcatalog','youtube','facebook']
num_paths = 80 # reproduction {walklets:1000,harp:40,n2v:10}
path_length = 40 # reproduction {walklets:11,harp:10,n2v:80}
dim = 128
workers = 18
window=10 
epochs=5
negative=5
sample=0.1
min_count=0
min_alpha=0.001 # reproduction {deepwalk:0.0001,n2v:0.0001}
alpha=0.025

for d in datasets:
    if d == 'youtube':
        num_paths = 10
        path_length = 80
    print("###", d, "###")
    print("## walking ##")
    # deepwalk random walk generation (same as n2v with p=q=1)
    G = graph.load_edgelist(data_path + d + '.edges')
    walks = graph.build_deepwalk_corpus(G, num_paths=num_paths, path_length=path_length) 
    print("done")
    
#     # n2v random walk generation (slower due to pre-processing of transition probs)
#     G = nx.read_edgelist(data_path + d + '.edges')
#     for edge in G.edges():
#             G[edge[0]][edge[1]]['weight'] = 1
#     nG = node2vec.Graph(G, False, 0.25, 0.25)
#     print('preprocessing')
#     nG.preprocess_transition_probs()
#     print('walking')
#     walks = nG.simulate_walks(num_paths, path_length)

    # random (initialize embeddings randomly, don't train)
    print("# random #")
    rand_model = Word2Vec(size=dim,min_count=0)
    rand_model.build_vocab(walks)
    rand_model.save_word2vec_format(emb_path + d + '.random.wv')
    
    # deepwalk
    print("# deepwalk #")
    dw = Word2Vec(walks, sg=1, size=dim, workers=workers, window=window, negative=negative, sample=sample, 
                  min_count=min_count, min_alpha=min_alpha, alpha=alpha, iter=epochs)
    dw.save_word2vec_format(emb_path + d + '.deepwalk.wv')
    
#     # node2vec
#     print("node2vec")
#     dw = Word2Vec(walks, sg=1, size=dim, workers=workers, window=window, negative=negative, sample=sample, 
#                   min_count=min_count, min_alpha=min_alpha, alpha=alpha, iter=epochs)
#     dw.save_word2vec_format(emb_path + d + '.node2vec.wv')
    
    # HARP
    print("# harp #")
    hp_graph = harp.load_graph(data_path + d + '.edges')
    harp.dw(hp_graph,iter_count=1,representation_size=dim, sfdp_path='./HP/SFDP/sfdp_linux', 
            num_walks=num_paths, walk_length=path_length,window_size=window,
           outfile=emb_path + d + '.harp.wv',workers=workers,hs=0,sg=1,
            negative=negative,
            sample=sample,
            min_count=min_count,
            min_alpha=min_alpha,
            alpha=alpha)

    # walklets
    print("# walklets #")
    walker = walklets.WalkletMachine(walklets.AttrDict({
        'input':data_path + d + '.edges',
        'output':emb_path + d + '.walklets.wv',
        'window_size':2, # skip-window size, not window size
        'dimensions':dim,
        'walk_number':num_paths,
        'walk_length':path_length,
        'workers':workers,
        'window':window,
        'negative':negative,
        'sample':sample,
        'min_count':min_count,
        'min_alpha':min_alpha,
        'alpha':alpha,
        'iter':epochs
    }))
    walker.walks = walks # re-use already created walks instead of walking again
    walker.create_embedding()
    walker.save_model()

    # HALK
    print("# halk #")
    percentages = [0.1,0.2,0.4,1]
    start_alphas = [0.025 for i in range(4)]
    end_alphas = [0.001 for i in range(4)]
    epochs_list = [10,5,3,1]
    halk_model = halk.embed(walks, percentages, start_alphas, end_alphas, epochs_list, dimensions=dim, workers=workers, 
                            window=window, negative=negative, sample=sample, min_count=min_count, shuffle=False)
    halk_model.save_word2vec_format(emb_path + d + '.halk.wv')

### cora ###
## walking ##
done 216640 of 216640
# random #
# deepwalk #
# harp #
coarsened to 18 levels
# walklets #

Optimization round: 1/2.
Creating documents.
Fitting model.

Optimization round: 2/2.
Creating documents.
Fitting model.

Models are integrated to be multi scale.
Saving to disk.
# halk #
training 1 of 4
training 2 of 4
training 3 of 4
training 4 of 4
### citeseer ###
## walking ##
done 262320 of 262320
# random #
# deepwalk #
# harp #
coarsened to 17 levels
# walklets #

Optimization round: 1/2.
Creating documents.
Fitting model.

Optimization round: 2/2.
Creating documents.
Fitting model.

Models are integrated to be multi scale.
Saving to disk.
# halk #
training 1 of 4
training 2 of 4
training 3 of 4
training 4 of 4
### blogcatalog ###
## walking ##
done 824960 of 824960
# random #
# deepwalk #
# harp #
coarsened to 24 levels
# walklets #

Optimization round: 1/2.
Creating documents.
Fitting model.

Optimization round: 2/2.
Creating documents.
Fitting model.

Models

# Node Classification

In [3]:
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

data_path = './data/processed/'
emb_path = './data/emb/'
datasets = ['cora','citeseer']
algos = ['halk','harp','walklets','deepwalk']
folds = 10
result = {a:{} for a in algos}

for d in datasets:
    print('### ' + d + ' ###')
    labels = defaultdict(str)
    with open(data_path + d + '.labels') as f:
        for line in f.readlines():
            line = line.split()
            labels[int(line[0])] = line[1]
    y = [labels[i] for i in labels]
    X_idx = [i for i in labels]
    for algo in algos:
        model = Word2Vec.load_word2vec_format(emb_path + d + '.' + algo + '.wv',binary=False)
        X = [model[str(i)] for i in X_idx]
        clf = LogisticRegression(solver='liblinear',multi_class='ovr')
#         clf = SVC(kernel='linear')
        scores = []
        for i in range(folds):
            X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=i)
            clf.fit(X_train,y_train)
            score = metrics.f1_score(y_test,clf.predict(X_test), average='macro')
            scores.append(score)
        print(algo, ' Macro F1: ', np.mean(scores), np.std(scores))  
        result[algo][d] = np.mean(scores)

### cora ###
halk  Macro F1:  0.816499559297 0.0217438766737
harp  Macro F1:  0.813318212421 0.0249751491136
walklets  Macro F1:  0.810979845501 0.0245962082751
deepwalk  Macro F1:  0.813328141553 0.0197573656122
### citeseer ###
halk  Macro F1:  0.569702103703 0.0324652377252
harp  Macro F1:  0.561647249213 0.0202259437483
walklets  Macro F1:  0.557359332314 0.0285476271785
deepwalk  Macro F1:  0.555448282517 0.0281922118646


## BlogCatalog (multi-label)

In [4]:
# Oracle providing the number of classes to predict (often used in the multiclass-evaluations for GE)
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert len(X) == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [5]:
%%capture --no-stdout

from sklearn.preprocessing import MultiLabelBinarizer


labels = defaultdict(list)
with open(data_path + 'blogcatalog.labels') as f:
    for line in f.readlines():
        line = line.split()
        labels[int(line[0])].append(int(line[1].strip())-1)
        
y = [labels[i] for i in labels]
X_idx = [i for i in labels]
mlb = MultiLabelBinarizer()
mlb.fit(y)

for algo in algos:
    model = Word2Vec.load_word2vec_format(emb_path + 'blogcatalog.' + algo + '.wv',binary=False)
    X = [model[str(x)] for x in X_idx]
    clf = TopKRanker(LogisticRegression(solver='liblinear'))
    clf.classes_ = mlb.classes_
    scores = []
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=i)
        y_train = mlb.transform(y_train)
        top_k = [len(l) for l in y_test]
        clf.fit(X_train,y_train)
        pred = clf.predict(X_test, top_k)
        score = metrics.f1_score(mlb.transform(y_test),mlb.transform(pred), average='macro')
        scores.append(score)
    print(algo, ' Macro F1: ', np.mean(scores),'std',np.std(scores))  
    result[algo]['blogcatalog'] = np.mean(scores)

halk  Macro F1:  0.276973558876 std 0.00378331742802
harp  Macro F1:  0.276454758688 std 0.00587755926758
walklets  Macro F1:  0.273101922431 std 0.00498511241567
deepwalk  Macro F1:  0.275659316928 std 0.00509703795037


In [6]:
for algo in result:
    print(algo, np.mean(list(result[algo].values())))

halk 0.554391740625
harp 0.550473406774
walklets 0.547147033415
deepwalk 0.548145246999


# Link prediction

In [7]:
from link_prediction import create_train_test_graphs
from link_prediction import test_edge_functions, default_params, edge_functions
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec


for d in ['blogcatalog','facebook']:
    print('### ' + d + '###')
    Gtrain, Gtest = create_train_test_graphs(data_path + d + '.edges')
    edges_train, labels_train = Gtrain.get_selected_edges()
    edges_test, labels_test = Gtest.get_selected_edges()
    for algo in algos:
        print('## ' + algo + '##')
        auc = test_edge_functions(Gtrain,Gtest,emb_size=dim,
                                  model=Word2Vec.load_word2vec_format(emb_path + d + '.' + algo + '.wv',binary=False),
                                 edges_train=edges_train,
                                 edges_test=edges_test,
                                 labels_train=labels_train,
                                 labels_test=labels_test,
                                 lin_clf=LogisticRegression(solver='liblinear'))
        print(auc)

### blogcatalog###
Regenerating link prediction graphs
Read graph, nodes: 10312, edges: 333983
Finding 166991 of 52829533 non-edges
Finding 166991 positive edges of 333983 total edges
Found: 166990    
Read graph, nodes: 10312, edges: 333983
Finding 166991 of 52829533 non-edges
Finding 166991 positive edges of 333983 total edges
Found: 166990    
## halk##
{'hadamard': [0.82145006842727375], 'average': [0.93543304252414039], 'l1': [0.94900407653141827], 'l2': [0.9529188042503911]}
## harp##
{'hadamard': [0.78985976002940261], 'average': [0.93992224741446528], 'l1': [0.97485697633860879], 'l2': [0.97726696684512571]}
## walklets##
{'hadamard': [0.83649043811184476], 'average': [0.91279569597065624], 'l1': [0.98425703459862968], 'l2': [0.98544421458238207]}
## deepwalk##
{'hadamard': [0.80547420395904079], 'average': [0.9435172953696791], 'l1': [0.97858692068622199], 'l2': [0.98051736597870942]}
### facebook###
Regenerating link prediction graphs
Read graph, nodes: 4039, edges: 88234
Fin

# Shortest Path Approximation
## Create train/test (landmarks)

In [8]:
import landmarks as lm
import pickle


train,test = lm.generate_data('./data/processed/youtube.edges', k_train=5, k_test=2)
with open('./data/train_youtube.pkl', 'wb') as f:
    pickle.dump(train,f)
    
with open('./data/test_youtube.pkl', 'wb') as f:
    pickle.dump(test,f)
    
train,test = lm.generate_data('./data/processed/facebook.edges', k_train=100, k_test=100)
with open('./data/train_facebook.pkl', 'wb') as f:
    pickle.dump(train,f)  
with open('./data/test_facebook.pkl', 'wb') as f:
    pickle.dump(test,f)

train set - all nodes 1134890, remaining nodes after selecting training landmarks 1134885
test set path pairs for 5 of 5 landmarks
shortest path pairs for 2 of 2 landmarks
11348844 training, 4539528 test pairs
train set - all nodes 4039, remaining nodes after selecting training landmarks 3939
test set path pairs for 100 of 100 landmarks
shortest path pairs for 100 of 100 landmarks
779302 training, 759210 test pairs


## Approximate paths

In [6]:
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.special import expit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

def prepare_binary_ops(raw_data, emb_model, op='avg'):
    x = []
    y = []
    for k,v in raw_data.items():
        emb1,emb2 = emb_model[str(k[0])],emb_model[str(k[1])]
        if op == 'sub':
            x.append(emb1-emb2)
        elif op == 'abs':
            x.append(abs(emb1-emb2))
        elif op == 'avg':
            x.append((emb1+emb2)/2)
        elif op == 'had':
            x.append(emb1*emb2)
        elif op == 'cc':
            x.append(np.concatenate((emb1,emb2)))
        elif op == 'dist':
            x.append([euclidean(emb1,emb2)])
        elif op == 'smax':
            x.append([expit(np.dot(emb1,emb2))])
        else:
            x.append([np.dot(emb1,emb2)])
        y.append(v)
    return x,y      

# loop over datasets (facebook, youtube)
for d in ['facebook','youtube']:
    print('### ' + d + '###')
    baseline_length = 2
    for algo in algos + ['random']:
        print('## ' + algo + '##')
        print('\t MAE \t\t\t MRE')
        train_t = pickle.load(open('./data/train_' + d + '.pkl','rb'))
        test_t = pickle.load(open('./data/test_' + d + '.pkl','rb'))
        model = Word2Vec.load_word2vec_format(emb_path + d + '.' + algo + '.wv',binary=False)
        for operator in ['abs', 'had','sub', 'avg', 'cc']:
            train_x,train_y = prepare_binary_ops(train_t, model, op=operator)
            test_x, test_y = prepare_binary_ops(test_t, model, op=operator)  

            clf = LinearRegression()
            clf.fit(train_x, train_y)
            pred = np.array([round(p) for p in clf.predict(test_x)])
            test_y = np.array(test_y)
    
            mre = np.mean(np.abs(pred-test_y)/test_y)
            print(operator, '\t', mean_absolute_error(test_y,pred),'\t',mre)
        baseline = np.ones_like(pred)*baseline_length
        mre = np.mean(np.abs(baseline-test_y)/test_y)
        print('base',baseline_length, '\t', mean_absolute_error(test_y,baseline),'\t\t',mre)
        baseline_length += 1

### facebook###
## halk##
	 MAE 			 MRE
abs 	 0.399209704825 	 0.115247951975
had 	 0.419025039185 	 0.119112326033
sub 	 0.906753072272 	 0.30628479102
avg 	 0.683299745788 	 0.225088541284
cc 	 0.683303697264 	 0.225082661112
base 2 	 1.77639915175 		 0.409245010948
## harp##
	 MAE 			 MRE
abs 	 0.429975895997 	 0.121595489923
had 	 0.50913186075 	 0.150667996997
sub 	 0.906753072272 	 0.30628479102
avg 	 0.675947366341 	 0.224864564732
cc 	 0.676013224273 	 0.224868321771
base 3 	 1.1094295386 		 0.280382709847
## walklets##
	 MAE 			 MRE
abs 	 0.416954465826 	 0.118550366578
had 	 0.648742772092 	 0.184618525773
sub 	 0.906753072272 	 0.30628479102
avg 	 0.668394778783 	 0.221494263147
cc 	 0.668390827307 	 0.22149669989
base 4 	 0.906753072272 		 0.30628479102
## deepwalk##
	 MAE 			 MRE
abs 	 0.45820260534 	 0.130860446416
had 	 0.414625729377 	 0.120915522464
sub 	 0.906753072272 	 0.30628479102
avg 	 0.683452536189 	 0.226944938379
cc 	 0.683447267554 	 0.226943787433
base 5 	 